In [3]:
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from google.colab import drive, files
drive.mount('/content/drive')
rootpath = '/content/drive/Shareddrives/KW phase1 repo'

import sys
sys.path.append(rootpath+'/package')
from utils.env import env

Requirement already up-to-date: Ipython in /usr/local/lib/python3.7/dist-packages (7.25.0)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import math
import numpy as np

In [15]:
class PathLossModel():
  def __init__(self):
    self._refDistance = env.REFERENCE_DISTANCE
    self._refPL = env.REFERENCE_PL
    self._noiseMaker = 0
    self._ple = env.PLE

  def estimateRSS(self, txPower, distance):
    return txPower-self._refPL+self._noiseMaker.noise()-(10*2*math.log10(distance/self._refDistance))

  def estimateDistance(self, txPower, rss):
    return self._refDistance*math.pow(10, ((txPower-self._refPL) - rss)/(10*(self._ple)))

  @property
  def noise(self):
    return self._noiseMaker.noise()

  @property
  def ple(self):
    return self._ple

  @property
  def noiseMaker(self):
    return self._noiseMaker

  @noise.setter
  def noiseMaker(self, noiseMaker):
    self._noiseMaker = noiseMaker

  @ple.setter
  def ple(self, ple):
    self._ple = ple

In [10]:
class NoiseMaker():
	def __init__(self, mean = 0, deviation = 1):
		self._mean = mean
		self._deviation = deviation

	def noise(self):
		return np.random.normal(self._mean, self._deviation)

In [18]:
if __name__ == '__main__':
  pathLossModel = PathLossModel()
  pathLossModel.noiseMaker = NoiseMaker(mean = env.noiseMean, deviation = env.noiseDeviation)

  rss = pathLossModel.estimateRSS(txPower=env.UAV_TRANSMISSION_POWER, distance=20)
  print("Distance {} m -> RSS: {} dBm".format(20, rss))
  d = pathLossModel.estimateDistance(txPower=env.UAV_TRANSMISSION_POWER, rss=rss)
  print("RSS: {} dBm -> Distance: {} m".format(rss, d))

Distance 20 m -> RSS: -46.023970483260584 dBm
RSS: -46.023970483260584 dBm -> Distance: 20.007762530225268 m
